In [2]:
import numpy as np
import pandas as pd
from pathlib import Path

In [5]:
result = pd.read_parquet('data/product_images.parquet', engine='pyarrow')

In [6]:
result

,asin,title,primary_image
0,B0B7NV73PJ,SanDisk 256GB Ultra microSDXC UHS-I Memory Car...,https://m.media-amazon.com/images/I/41MpKnSAd0...
1,B0B7NVXLLM,SanDisk 512GB Ultra microSDXC UHS-I Memory Car...,https://m.media-amazon.com/images/I/41ikVLl+gp...
2,B0B985Q9F1,SanDisk 256GB microSDXC -Card Licensed for Nin...,https://m.media-amazon.com/images/I/51X0sUzOQ7...
3,B071FVF3T7,Zinus 6 Inch Green Tea Memory Foam Mattress / ...,https://m.media-amazon.com/images/I/51t3X6VyK+...
4,B00Q7EPY04,Zinus 12 Inch Green Tea Memory Foam Mattress /...,https://m.media-amazon.com/images/I/51tBWhJnUF...
...,...,...,...
99995,B07R3K2F94,QuiFit Motivational Gallon Water Bottle - with...,https://m.media-amazon.com/images/I/516qh9MKbC...
99996,B084GRZGBN,Leemax VibroComf Neck Pillow for Traveling Was...,https://m.media-amazon.com/images/I/41v2nairg4...
99997,B08MTSW5KB,FESHFEN Invisible Wire Hair Extensions with Tr...,https://m.media-amazon.com/images/I/41AGbpZHAj...
99998,B094DL8WD7,Gua Sha Massage Tools Set Aventurine Natural g...,https://m.media-amazon.com/images/I/312VuAOYZl...


In [7]:
result['primary_image'][99998]

'https://m.media-amazon.com/images/I/312VuAOYZlS._SL500_.jpg'

In [ ]:
clip.available_models()

In [5]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)


Label probs: [[0.13612436 0.44617712 0.4176985 ]]


In [9]:

image = preprocess(Image.open("data/images/B00AIE2SOO.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
  #  logits_per_image, logits_per_text = model(image, text)
#    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

#print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]
print(image_features.shape)
print(text_features.shape)

torch.Size([1, 768])
torch.Size([3, 768])


['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [ ]:
torch.save(model, f"CLIP_model.pt") 

In [1]:
from  dataset import Amazon_products
import requests
import torch
import clip
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from multiprocessing import Pool
import numpy as np
import pandas as pd

#importing model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14@336px", device=device)





In [2]:


class Amazon_products(Dataset):
    """Amazon_products dataset."""

    def __init__(self, parquet_file, root_dir, transform=None):
        """
        Arguments:
            parquet_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = pd.read_parquet(parquet_file, engine='pyarrow')
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # get the data from the dataframe
        url = self.dataframe['primary_image'].iloc[idx]
        asin= self.dataframe['asin'].iloc[idx]
        description= self.dataframe['title'].iloc[idx]

        #get image from url
        response = requests.get(url)
        with open(str(asin)+'.jpg', 'wb') as image:
          image.write(response.content) 

        #open the image
        image= Image.open(str(asin)+'.jpg')
        plt.imshow(image)
        sample = {'image': image, 'asin': asin, 'description':description, 'url': url}

        #apply transormation to tensor
        if self.transform:
             transform = transforms.Compose([ transforms.ToTensor()])
             sample = {'tensor':transform(image), 'image': image, 'asin': asin, 'description':description, 'url': url}
             return sample


        return sample


In [11]:

def emmbed_images(url,asin):
    # get image from url
    response = requests.get(url)
    with open(str(idx)+'.jpg', 'wb') as handle:
        handle.write(response.content)
    image= Image.open(str(asin)+'.jpg')
    print("got image")
    #get encoding
    image = preprocess(image).unsqueeze(0).to(device)
    image_features = model.encode_image(image)
    return {'asin':asin, 'image_features':image_features}


In [4]:


amazon_dataset = Amazon_products(parquet_file= 'data/product_images.parquet', root_dir='data/images/', transform = True)
dataloader = DataLoader(amazon_dataset, batch_size=128, shuffle=True, num_workers=4)
#for i_batch, sample_batched in enumerate(dataloader):
 #   pass


pool = Pool()
urls= amazon_dataset.dataframe['primary_image'][0:10]
idxs=amazon_dataset.dataframe['asin'][0:10]
tuples = zip(urls,idxs)
dicts= pool.starmap(emmbed_images, tuples)

df = pd.DataFrame.from_dict(dicts)    
print(df)

Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Process SpawnPoolWorker-6:
Process SpawnPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.16/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python@3.9/3.9.16/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python@3.9/3.9.16/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python@3.9/3.9.16/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python@3.9/3.9.16/Frameworks/Python.framework

KeyboardInterrupt: 

In [4]:
amazon_dataset = Amazon_products(parquet_file= 'data/product_images.parquet', root_dir='data/images/', transform = True)
dataloader = DataLoader(amazon_dataset, batch_size=128, shuffle=True, num_workers=4)
ls=[]


In [8]:
for i in range(1):
    url= amazon_dataset.dataframe['primary_image'][i]
    idx=amazon_dataset.dataframe['asin'][i]
    ls.append(emmbed_images(url,idx))

df = pd.DataFrame.from_dict(ls)    
print(df)

KeyboardInterrupt: 

In [9]:
url= amazon_dataset.dataframe['primary_image'][0]
idx=amazon_dataset.dataframe['asin'][0]
print(url,idx)


https://m.media-amazon.com/images/I/41MpKnSAd0L._SL500_.jpg B0B7NV73PJ


In [12]:
emmbed_images(url,idx)

got image


{'asin': 'B0B7NV73PJ',
 'image_features': tensor([[ 2.7130e-01,  6.6387e-01, -2.1688e-02, -9.9078e-02,  6.9351e-01,
           2.4134e-01,  2.6439e-01,  4.6732e-02, -2.5134e-02,  1.0027e-01,
          -3.8436e-01, -3.3270e-01, -3.1529e-02,  5.9548e-01, -4.2292e-01,
          -2.3099e-01,  2.8239e-02,  9.4160e-01,  2.1856e-01,  4.7258e-01,
           8.0078e-02,  4.8320e-01,  4.5303e-01,  7.8132e-01, -6.6287e-01,
          -2.9959e-01, -2.7467e-01, -4.3766e-01, -5.8728e-01,  2.3417e-01,
          -1.0588e+00,  5.7656e-01, -7.0221e-01, -1.1358e-01,  4.1726e-02,
           3.5063e-01, -5.5660e-02,  4.7532e-01,  5.3285e-01, -9.1757e-01,
           1.3517e-01, -6.6431e-01,  1.7108e-01,  2.1420e-01,  4.6383e-01,
           1.6479e-01, -1.8569e-01, -3.5704e-02,  2.0597e-01,  2.2987e-01,
          -2.4792e-01,  5.0065e-01, -2.7522e-01,  6.5642e-02, -7.3230e-02,
          -1.5222e-01,  1.2525e+00, -4.7290e-01,  1.8077e-01,  1.5000e-01,
          -4.4674e-01, -1.1299e-01, -7.9875e-02,  2.7807e-0